In [41]:
import pandas as pd
import requests
import re
import time

In [26]:
#reading in csv and selecting only the first artist of each song 
df = pd.read_csv("cleaned_spotify_data.csv",usecols=list(range(21)))
df.head()

,valence,year,acousticness,artists_combined,danceability,duration_ms,energy,explicit,id,instrumentalness,...,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,artist_count,artist_1
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,...,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921-01-01,0.0366,80.954,3,Sergei Rachmaninoff
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,...,0.160,-12.441,1,Clancy Lowered the Boom,5,1921-01-01,0.4150,60.936,1,Dennis Day
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,...,0.101,-14.850,1,Gati Bali,5,1921-01-01,0.0339,110.339,1,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,...,0.381,-9.316,1,Danny Boy,3,1921-01-01,0.0354,100.109,1,Frank Parker
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,...,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921-01-01,0.0380,101.665,1,Phil Regan


In [170]:

def extract_genre(artist):
    if isinstance(artist, str):
        formatted_name = artist.replace(' ', '_')
        url = 'https://en.wikipedia.org/w/api.php'
        params = {
            'action': 'query',
            'format': 'json',
            'titles': formatted_name,
            'prop': 'revisions',
            'rvprop': 'content',
            'rvsection': 0,  # This is where the infobox is located
            'rvslots': 'main'
        }

        # initialize retries count
        retries = 3
        delay = 5  # seconds

        for attempt in range(retries):
            try:
                response = requests.get(url, params=params, timeout=10)  # 10 sec timeout
                data = response.json()
                matched_string = re.findall(r"genre\s*=\s*\{\{(hlist|flatlist)\|([^\}]*)\}\}", str(data))
                if matched_string:
                    genres_string = " ".join(match[1] for match in matched_string)
                    genres = re.findall(r"\[\[.*?\|(.*?)\]\]", genres_string)
                    return genres
                else:
                    return None
            except requests.exceptions.Timeout:
                if attempt < retries - 1:  # if we have retries left,
                    time.sleep(delay)  # wait before retrying
                    continue
                else:  # no retries left
                    return None  # could not fetch the genre
            except requests.exceptions.RequestException as e: 
                print(f"Request failed: {e}")
                return None
    else:
        # if the artist is not a string, return None 
        return None


In [121]:
#Tests
# artists = ['Pink Floyd', 'The Beatles', 'Led Zeppelin','Justin Bieber']
# df_test = pd.DataFrame(artists, columns=['artist_1'])
# df_test['genres'] = df_test['artist_1'].apply(extract_genre)
# df_test2 = df[:30]
# df_test2['genres'] = df_test2['artist_1'].apply(extract_genre)

# 16 seconds for 100 rows
#df_test3 = df.copy()[:100]
#df_test3['genres'] = df_test3['artist_1'].apply(extract_genre)

# 2 min 40 sec for 1000 rows
# df_test4 = df.copy()[:1000]
# df_test4['genres'] = df_test4['artist_1'].apply(extract_genre)



In [175]:
# Applying function to our df
# 7.5 - 8 hours
df['genres'] = df['artist_1'].apply(extract_genre)

In [195]:
df2_post = df.copy()
df2_post.head()

,valence,year,acousticness,artists_combined,danceability,duration_ms,energy,explicit,id,instrumentalness,...,loudness,mode,name,popularity,release_date,speechiness,tempo,artist_count,artist_1,genres
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,...,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921-01-01,0.0366,80.954,3,Sergei Rachmaninoff,None
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,...,-12.441,1,Clancy Lowered the Boom,5,1921-01-01,0.4150,60.936,1,Dennis Day,None
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,...,-14.850,1,Gati Bali,5,1921-01-01,0.0339,110.339,1,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...,None
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,...,-9.316,1,Danny Boy,3,1921-01-01,0.0354,100.109,1,Frank Parker,None
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,...,-10.096,1,When Irish Eyes Are Smiling,2,1921-01-01,0.0380,101.665,1,Phil Regan,None


In [252]:
print("Portion of song names with genres:", str(round(len(df2_post[~df2_post['genres'].isnull()])/len(df2_post)*100,1)) + '%', "=",len(df2_post[~df2_post['genres'].isnull()]),"rows")

Portion of song names with genres: 33.8% = 57672 rows


In [ ]:
#pulling all non-null values into a new df
df2_filtered = df2_post[~df2_post['genres'].isnull()]

In [211]:
# Find the maximum number of genres 
max_genres = int(df2_post['genres'].str.len().max())

# create new columns for each genre
for i in range(max_genres):
    df2_post[f'genre_{i+1}'] = df2_post['genres'].str.get(i)

In [218]:
import re

def clean_genres(genre_list):
    if genre_list is None:
        return []
    
    clean_list = []
    for item in genre_list:
        item = re.sub(r"\[\[|\]\]", "", item)  
        item = re.sub(r"last1=.*", "", item)   
        item = item.split('|')[-1] if '|' in item else item  
        if item.strip():  #
            clean_list.append(item.strip())
    return clean_list

df2_filtered['clean_genres'] = df2_filtered['genres'].apply(clean_genres)



<ipython-input-218-bc07e4c075f0>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_filtered['clean_genres'] = df2_filtered['genres'].apply(clean_genres)


In [221]:

max_genres = df2_filtered['clean_genres'].str.len().max() 

for i in range(max_genres):
    df2_filtered[f'genre_{i+1}'] = df2_filtered['clean_genres'].apply(lambda genres: genres[i] if len(genres) > i else None)


<ipython-input-221-81fae8347d74>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_filtered[f'genre_{i+1}'] = df2_filtered['clean_genres'].apply(lambda genres: genres[i] if len(genres) > i else None)


In [227]:
df2_filtered = df2_filtered.drop('genres', axis=1)
df2_filtered = df2_filtered.drop('clean_genres', axis=1)
df2_filtered = df2_filtered.reset_index(drop = True)

In [237]:
df2_filtered.head()

,valence,year,acousticness,artists_combined,danceability,duration_ms,energy,explicit,id,instrumentalness,...,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11
0,0.826,1921,0.995,['Maurice Chevalier'],0.463,147133,0.260,0,0BMkRpQtDoKjcgzCpnqLNa,0.0,...,None,None,None,None,None,None,None,None,None,None
1,0.678,1921,0.996,['Maurice Chevalier'],0.500,181733,0.274,0,0i7MdVu0tNEyUdgpCBilKe,0.0,...,None,None,None,None,None,None,None,None,None,None
2,0.651,1921,0.992,['Maurice Chevalier'],0.522,170627,0.290,0,1RgXfAX9PrSfTKTFvOQxPR,0.0,...,None,None,None,None,None,None,None,None,None,None
3,0.767,1921,0.995,['Maurice Chevalier'],0.517,169173,0.271,0,39ZYlUAoqyE05y7rsfeivo,0.0,...,None,None,None,None,None,None,None,None,None,None
4,0.775,1921,0.996,['Maurice Chevalier'],0.526,153973,0.247,0,4wX1xtFyqlhBMBX7xaOfNB,0.0,...,None,None,None,None,None,None,None,None,None,None


In [249]:
df2_filtered.to_csv("Spotify Dataset - Genres Cleaned.csv", index = False)